In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as f
import torch.nn as nn
import struct
from torch.utils.tensorboard import SummaryWriter

In [2]:
class MNIST(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return X[idx], Y[idx]

In [3]:
def load_mnist_dataset(image_path, label_path):
    with open(image_path, mode='rb') as fd:
        magic_number = struct.unpack('>I', fd.read(4))[0]
        assert (magic_number == 2051)
        n = struct.unpack('>i', fd.read(4))[0]
        rows = struct.unpack('>i', fd.read(4))[0]
        cols = struct.unpack('>i', fd.read(4))[0]
        buffer = fd.read(n * rows * cols)
        X = torch.tensor(list(buffer), pin_memory=True).reshape(n, rows * cols)

    with open(label_path, mode='rb') as fd:
        eye = torch.eye(10).tolist()
        magic_number = struct.unpack('>I', fd.read(4))[0]
        assert (magic_number == 2049)
        n = struct.unpack('>i', fd.read(4))[0]
        labels = list(fd.read(n))
        Y = [eye[l] for l in labels]
    
    Y = torch.tensor(Y, pin_memory=True)
    
    X = X.type(torch.float).to('cuda')
    Y = Y.type(torch.float).to('cuda')
    
    X /= 255.0
    
    return X, Y

In [4]:
def train(model, trainset, valset, epoch):
    writer = SummaryWriter()
    optim = torch.optim.SGD(model.parameters(), lr = 0.01, weight_decay=0.001)
    train_loader = DataLoader(trainset, batch_size=16)
    best_val_acc = 0
    best_val_epoch = 0
    best_train_acc = 0
    best_train_epoch = 0
    
    for cur_epoch in range(epoch):
        train_tp = 0
        train_count = 0
        avg_loss = 0
        
        for x, label in train_loader:
            optim.zero_grad()
            pred = model(x)
            loss = f.mse_loss(pred, label)
            avg_loss = (avg_loss + loss.item()) / 2
            loss.backward()
            optim.step()
            
            with torch.no_grad():
                pred_class = pred.argmax(dim=1)
                label_class = label.argmax(dim=1)
                for p, l in zip(pred_class, label_class):
                    if(p.item() == l.item()):
                        train_tp += 1
                    train_count += 1
                    
        train_acc = train_tp / train_count
        if(train_acc > best_train_acc):
            best_train_acc = train_acc
            best_train_epoch = cur_epoch + 1
        
        val_loader = DataLoader(valset, batch_size=8)
        with torch.no_grad():
            val_tp = 0
            val_count = 0
            for x, label in val_loader:
                pred = model(x)
                pred_class = pred.argmax(dim=1)
                label_class = label.argmax(dim=1)
                
                for p, l in zip(pred_class, label_class):
                    if(p.item() == l.item()):
                        val_tp += 1
                    val_count += 1
            
            val_acc = val_tp / val_count
            if(val_acc > best_val_acc):
                best_val_acc = val_acc
                best_val_epoch = cur_epoch + 1

            writer.add_scalar('Train/Loss', avg_loss, cur_epoch)
            print(f'Epoch {cur_epoch + 1}: VAL_ACC = {best_val_acc:.3f}(at {best_val_epoch}), LOSS_AVG = {avg_loss:.3f}, TRAIN_ACC = {best_train_acc:.3f} (at {best_train_epoch})', end='\n')

In [5]:
torch.manual_seed(0)
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        
        self.seq = nn.Sequential(
            nn.Linear(28 * 28, 300),
            nn.ReLU(),
            nn.Linear(300, 10)
        )
    
    def forward(self, x):
        return self.seq(x)

In [6]:
X, Y = load_mnist_dataset('train-images-idx3-ubyte', 'train-labels-idx1-ubyte')
dataset = MNIST(X, Y)
trainset, valset = random_split(dataset, [0.8, 0.2])

In [7]:
model = Network().to('cuda')
train(model, trainset, valset, 100)

Epoch 1: VAL_ACC = 0.853(at 1), LOSS_AVG = 0.039, TRAIN_ACC = 0.766 (at 1)
Epoch 2: VAL_ACC = 0.883(at 2), LOSS_AVG = 0.030, TRAIN_ACC = 0.873 (at 2)
Epoch 3: VAL_ACC = 0.898(at 3), LOSS_AVG = 0.025, TRAIN_ACC = 0.893 (at 3)
Epoch 4: VAL_ACC = 0.907(at 4), LOSS_AVG = 0.022, TRAIN_ACC = 0.904 (at 4)
Epoch 5: VAL_ACC = 0.914(at 5), LOSS_AVG = 0.019, TRAIN_ACC = 0.911 (at 5)
Epoch 6: VAL_ACC = 0.918(at 6), LOSS_AVG = 0.018, TRAIN_ACC = 0.917 (at 6)
Epoch 7: VAL_ACC = 0.922(at 7), LOSS_AVG = 0.017, TRAIN_ACC = 0.921 (at 7)
Epoch 8: VAL_ACC = 0.925(at 8), LOSS_AVG = 0.016, TRAIN_ACC = 0.925 (at 8)
Epoch 9: VAL_ACC = 0.928(at 9), LOSS_AVG = 0.015, TRAIN_ACC = 0.928 (at 9)
Epoch 10: VAL_ACC = 0.930(at 10), LOSS_AVG = 0.014, TRAIN_ACC = 0.930 (at 10)
Epoch 11: VAL_ACC = 0.932(at 11), LOSS_AVG = 0.014, TRAIN_ACC = 0.932 (at 11)
Epoch 12: VAL_ACC = 0.933(at 12), LOSS_AVG = 0.013, TRAIN_ACC = 0.933 (at 12)
Epoch 13: VAL_ACC = 0.935(at 13), LOSS_AVG = 0.013, TRAIN_ACC = 0.935 (at 13)
Epoch 14: VAL